# EDA

####  Run this cell to set up and start your interactive session.


In [14]:
%idle_timeout 2880
%glue_version 5.0
%worker_type G.1X
%number_of_workers 5

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
from pyspark.sql.functions import month, to_date, col
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

You are already connected to a glueetl session 6065a663-d101-4a25-b138-3dc0dc030178.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Current idle_timeout is 2880 minutes.
idle_timeout has been set to 2880 minutes.


You are already connected to a glueetl session 6065a663-d101-4a25-b138-3dc0dc030178.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Setting Glue version to: 5.0


You are already connected to a glueetl session 6065a663-d101-4a25-b138-3dc0dc030178.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Previous worker type: G.1X
Setting new worker type to: G.1X


You are already connected to a glueetl session 6065a663-d101-4a25-b138-3dc0dc030178.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Previous number of workers: 5
Setting new number of workers to: 5



#### Example: Create a DynamicFrame from a table in the AWS Glue Data Catalog and display its schema


In [6]:
dyf = glueContext.create_dynamic_frame.from_catalog(database='proyecto1db', table_name='1')
dyf.printSchema()

root
|-- armas_medios: string
|-- departamento: string
|-- municipio: string
|-- fecha_hecho: string
|-- GENERO: string
|-- AGRUPA_EDAD_PERSONA: string
|-- CANTIDAD: int
|-- id: string


#### Example: Convert the DynamicFrame to a Spark DataFrame and display a sample of the data


In [9]:
df = dyf.toDF()
# Reemplazamos los espacios en blanco y convertimos a minusculas
df = df.toDF(*[c.strip().replace(" ", "_").lower() for c in df.columns])
df.createOrReplaceTempView("hurto")
df.show()

+-------------------+------------+--------------------+-----------+---------+-------------------+--------+--------------------+
|       armas_medios|departamento|           municipio|fecha_hecho|   genero|agrupa_edad_persona|cantidad|                  id|
+-------------------+------------+--------------------+-----------+---------+-------------------+--------+--------------------+
|SIN EMPLEO DE ARMAS|   ATLANTICO|       Barranquilla | 20/02/2025| FEMENINO|            ADULTOS|       1|99338118-0a96-4d3...|
|SIN EMPLEO DE ARMAS|CUNDINAMARCA|        Bogotá D.C. | 19/01/2025| FEMENINO|            ADULTOS|       1|99338118-0a96-4d3...|
|SIN EMPLEO DE ARMAS|CUNDINAMARCA|        Bogotá D.C. | 19/01/2025| FEMENINO|            ADULTOS|       1|99338118-0a96-4d3...|
|SIN EMPLEO DE ARMAS|CUNDINAMARCA|        Bogotá D.C. | 19/01/2025| FEMENINO|            ADULTOS|       1|99338118-0a96-4d3...|
|SIN EMPLEO DE ARMAS|CUNDINAMARCA|        Bogotá D.C. | 19/01/2025| FEMENINO|            ADULTOS|       

In [16]:
df.select("fecha_hecho").show(10, truncate=False)

df = df.withColumn("fecha_hecho", to_date(col("fecha_hecho"), "dd/MM/yyyy"))

df.createOrReplaceTempView("hurto")



+-----------+
|fecha_hecho|
+-----------+
|2025-02-20 |
|2025-01-19 |
|2025-01-19 |
|2025-01-19 |
|2025-01-19 |
|2025-01-12 |
|2025-01-19 |
|2025-01-01 |
|2025-01-05 |
|2025-02-08 |
+-----------+
only showing top 10 rows


In [17]:
resultado = spark.sql("""
    SELECT year(fecha_hecho)  AS anio,
           month(fecha_hecho) AS mes,
           COUNT(*)           AS casos
    FROM hurto
    GROUP BY anio, mes
    ORDER BY anio, mes
""")

resultado.show()

+----+---+-----+
|anio|mes|casos|
+----+---+-----+
|2025|  1|20959|
|2025|  2|20351|
|2025|  3|21749|
+----+---+-----+


In [18]:
resultado = spark.sql("""
    SELECT municipio, COUNT(*) AS casos
    FROM hurto
    GROUP BY municipio
    ORDER BY casos DESC
    LIMIT 10;
""")

resultado.show()

+--------------+-----+
|     municipio|casos|
+--------------+-----+
|  Bogotá D.C. |30955|
|     Medellín | 4262|
| Barranquilla | 2671|
|    Cartagena | 1727|
|        Pasto | 1419|
|  Bucaramanga | 1306|
|Villavicencio | 1232|
|        Soacha| 1132|
|  Santa Marta | 1055|
|        Neiva | 1037|
+--------------+-----+


In [19]:
resultado = spark.sql("""
    -- Serie mensual 2024-2025
    SELECT date_format(fecha_hecho,'yyyy-MM') AS mes,
           SUM(cantidad)                      AS casos
    FROM hurto
    GROUP BY mes
    ORDER BY mes;
""")

resultado.show()

+-------+-----+
|    mes|casos|
+-------+-----+
|2025-01|20959|
|2025-02|20351|
|2025-03|21749|
+-------+-----+


In [20]:
resultado = spark.sql("""
    -- Por departamento
    WITH tmp AS (
      SELECT year(fecha_hecho)  AS anio,
             departamento,
             COUNT(*)           AS casos
      FROM hurto
      GROUP BY anio, departamento
    )
    SELECT * FROM tmp
    ORDER BY anio, casos DESC;
""")

resultado.show()

+----+------------------+-----+
|anio|      departamento|casos|
+----+------------------+-----+
|2025|      CUNDINAMARCA|34383|
|2025|         ANTIOQUIA| 6340|
|2025|         ATLANTICO| 3658|
|2025|         SANTANDER| 2175|
|2025|           BOLIVAR| 2085|
|2025|            NARINO| 1928|
|2025|              META| 1556|
|2025|             HUILA| 1539|
|2025|            TOLIMA| 1323|
|2025|         MAGDALENA| 1312|
|2025|             CAUCA| 1131|
|2025|            BOYACA|  796|
|2025|             CESAR|  783|
|2025|NORTE DE SANTANDER|  762|
|2025|         RISARALDA|  707|
|2025|           CORDOBA|  523|
|2025|           QUINDIO|  495|
|2025|          CASANARE|  470|
|2025|            CALDAS|  320|
|2025|             SUCRE|  296|
+----+------------------+-----+
only showing top 20 rows


In [22]:
# Creamos un pivot para ver la cantidad de casos por mes y por armas_medios
# (armas_medios es una columna que tiene el tipo de arma o medio usado en el hurto)
pivot = (df.withColumn("mes", month("fecha_hecho"))
           .groupBy("mes")
           .pivot("armas_medios")
           .count()
           .orderBy("mes"))

pivot.show(10, truncate=False)

+---+---------------------------+-------------+------------+------------+-------------+------------+-------------------+
|mes|ARMA BLANCA / CORTOPUNZANTE|ARMA DE FUEGO|CONTUNDENTES|ESCOPOLAMINA|LLAVE MAESTRA|NO REPORTADO|SIN EMPLEO DE ARMAS|
+---+---------------------------+-------------+------------+------------+-------------+------------+-------------------+
|1  |3219                       |3159         |588         |110         |3            |769         |13111              |
|2  |3295                       |2657         |489         |129         |null         |762         |13019              |
|3  |3502                       |2917         |445         |108         |4            |835         |13938              |
+---+---------------------------+-------------+------------+------------+-------------+------------+-------------------+


In [23]:
pivot.write.mode("overwrite").format("parquet").save("s3://ds-trabajo1-parte2/refined/EDA/")